# Exercise 9: Principal and Independent Component Analysis (PCA & ICA)

**Note**: Please insert the names of all participating students:
1. 
2. 
3. 

## Preamble
The following code downloads and imports all necessary files and modules into the virtual machine of Colab. Please make sure to execute it before solving this exercise. This mandatory preamble will be found on all exercise sheets.

In [ ]:
import sys, os
if 'google.colab' in sys.modules:
  if os.getcwd() == '/content':
    !git clone 'https://github.com/inb-uni-luebeck/cs4405.git'
    os.chdir('cs4405')

import numpy as np
from matplotlib import pyplot as plt
from utils import utils_9 as utils

## Exercise 9.1: Principal Component Analysis (PCA)

*Principal Component Analysis (PCA)* is a procedure that uses an orthogonal transformation to convert a set of observations of possibly correlated variables into a set of values of linearly uncorrelated variables called principal components.

Let $\boldsymbol{X} \in \mathbb{R}^{L \times d}$ be a given data matrix that contains $L$ `samples` $\boldsymbol{x}_{i} \in \mathbb{R}^{1 \times d}$, $i = 1,...,L$.
The principal components of $\boldsymbol{X}$ can be derived by computing the `eigenvectors` $\boldsymbol{W} \in \mathbb{R}^{d \times d}$ and `eigenvalues` $\boldsymbol{v} \in \mathbb{R}^{1 \times d}$ of the sample `covariance` matrix

$$
\begin{equation}
    \boldsymbol{C}_{\boldsymbol{X}} = \frac{1}{L-1} \sum_{i=1}^{L} \left(\boldsymbol{x}_{i} - \boldsymbol{\mu}_{\boldsymbol{X}}\right)^{T} \left( \boldsymbol{x}_{i} - \boldsymbol{\mu}_{\boldsymbol{X}} \right)
\end{equation},
$$

with the `sample_mean` $\boldsymbol{\mu}_{\boldsymbol{X}} \in \mathbb{R}^{1 \times d}$

$$
\begin{equation}
    \boldsymbol{\mu}_{\boldsymbol{X}} = \frac{1}{L}\sum_{i=1}^{L} \boldsymbol{x}_{i}
\end{equation}.
$$

By performing a change of basis, the mean-free samples $\boldsymbol{\tilde{X}}$ can be represented within the obtained (principal component) basis $\boldsymbol{W}$ as follows:

$$
\begin{equation}
  \boldsymbol{\hat{X}} = \boldsymbol{\tilde{X}} \boldsymbol{W}
\end{equation}.
$$

**Tasks**:
- Implement the function `principal_component_analysis` that computes the principal components of a given RGB color image and represents the image within the obtained (principal component) basis.

**Programming Hints**:
- An RGB image $\boldsymbol{I} \in \mathbb{R}^{h \times w \times 3}$ can also be represented as $\boldsymbol{I'} \in \mathbb{R}^{h \cdot w \times 3}$ by flattening the first two dimensions.
- The first principal component corresponds to the `eigenvector` with the largest `eigenvalue`.
- The NumPy functions `np.linalg.eig` and `np.argsort` might by helpful.

**Questions**:
1. What is the relation between the `eigenvalues` and the marginal sample variances after the transformation?
2. Can you think of a reasonable interpretation of the image "color" channels after the transformation?

**Answers**:
1. 
2. 


In [ ]:
def principal_component_analysis(image):
  
  # store the image shape
  shape = image.shape

  # TODO: flatten the spatial dimensions of the image
  samples = 

  # n_samples: number of samples / n_features: number of features
  n_samples, n_features = samples.shape

  # TODO: subtract the sample mean
  samples = 

  # TODO: calculate the covariance matrix
  covariance = 

  # TODO: calculate the eigenvalues and eigenvectors (principal components)
  eigenvalues, eigenvectors = 

  # TODO: sort the eigenvectors based on the eigenvalues in descending order
  eigenvectors = 

  # TODO: perform a change of basis
  samples = 

  return np.reshape(samples, shape)


_, image = utils.load_data('data/data_9.npz')
utils.show_image(image)
image_pca = principal_component_analysis(image)
utils.show_image(image_pca)

## Exercise 9.2: Independent Component Analysis (ICA)
This exercise considers the *Independent Component Analysis (ICA)*. While the principal components in PCA are only uncorrelated, the independent components in ICA are also statistically independent.

To perform the ICA the following steps are necessary:

1. Compute and subtract the `sample_mean` of the `samples`.
2. Remove correlations from the data by performing a `principal_component_analysis` and a change of basis.
3. Now, correlations have been removed from the data but the dimensions do not have the same variance. In order to align the variances of the dimensions, each dimension is divided by its standard deviation. This step is also called *whitening*.
4. The last step tries to maximize the statistical independence of the data dimensions by rotating the whitened data $\boldsymbol{X} \in \mathbb{R}^{L \times d}$ as follows: 
$$
\begin{equation}
  \boldsymbol{Y} = \boldsymbol{X} \boldsymbol{A}_{\theta}^{T}
\end{equation},
$$
where
$$
\begin{equation}
    \boldsymbol{A}_{\theta} = \left(
        \begin{matrix}
            \cos(\theta) & -\sin(\theta) \\
            \sin(\theta)  & \cos(\theta)
        \end{matrix}
    \right)
\end{equation}
$$
is a `rotation_matrix` w.r.t. an `angle` $\theta$, such that the non-Gaussianity of the marginal data distributions is maximal, i.e., until the marginal distributions are as dissimilar to a Gaussian distribution as
possible. Utilize the `kurtosis` to measure the `non_gaussianity`, i.e., (iteratively) find the rotation
$$
\begin{equation}
    \theta^{*} = \underset{\theta}{\operatorname{argmax}}\left(
        \left\vert 
            3 - \frac{1}{d} \sum_{i=1}^{d} \text{kurtosis}(\boldsymbol{y}_i)
        \right\vert
    \right)
\end{equation}.
$$
where $\boldsymbol{y}_{i} \in \mathbb{R}^{1 \times L}$ is the i-th column of $\boldsymbol{Y}$.

Often ICA is used in order to perform *Blind Source Separation (BSS)*: Given a linear mixture of source signals (e.g. audio, images), BSS aims to reconstruct the original source signals from the recorded mixtures. This problem also is known as the *Cocktail Party Problem*, i.e., we want to obtain the separated statements of all participants of a recorded cocktail party.

**Tasks**:
- Implement the function `independent_component_analysis` that tries to reconstruct the original images from the mixture of two grayscale images.

**Programming Hints**:
- Reuse the function `principal_component_analysis` from exercise 9.1, which returns an uncorrelated image $\boldsymbol{I} \in \mathbb{R}^{h \times w \times 3}$ with zero mean.
- The functions `utils.rotation_matrix` and `utils.kurtosis` might be helpful.
- Use broadcasting and try to avoid for-loops.

**Questions**:
- Why do we look for the rotation that leads to maximal non-Gaussianity in the resulting data dimensions?

**Answers**:
- 


In [ ]:
def independent_component_analysis(image):
  
  # mean subtraction, pca, and change of basis
  image_pca = principal_component_analysis(image)

  # TODO: perform the whitening
  image_whitened = 

  # maximize the statistical independence
  max_non_gaussianity = 0
  for angle in range(180):
    
    # TODO: calculate the rotation matrix
    rotation_matrix = 

    # TODO: rotate the whitened image
    image_rotated = 

    # calculate the kurtosis of the rotated image
    kurtosis = 

    # calculate the non-gaussianity
    non_gaussianity = 
        
    if non_gaussianity > max_non_gaussianity:
      max_non_gaussianity = non_gaussianity
      image_ica = image_rotated

  return image_ica


image_1, image_2 = utils.load_data('data/data_9.npz',
                                   grayscale=True)
image = utils.mix_images(image_1, image_2)
utils.show_image(image)
image_ica = independent_component_analysis(image)
utils.show_image(image_ica)